In [1]:
from common.imports import *
from common.util import *
from common.const import *
from data.dataloader import *
from model.transformer import *
from model.run_model import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.empty_cache()

cuda


In [3]:
# PARAMS


In [4]:
# OBJECTS

## MODEL
model = stonks_transformer_model(
    d_model=D_MODEL,
    nhead=NUM_HEADS,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    num_decoder_layers=NUM_DECODER_LAYERS,
    dropout=DROPOUT
).to(device)
checkpoint = torch.load("best_model.pt")
model.load_state_dict(checkpoint['state_dict'])

## DATASET
dataset=ForecasterDataset(0, None)

In [5]:
# CHOOSE TICKER + DAYS_PRED
ticker = "TSLA"
days_pred = 5

In [6]:

idx = HISTORY_DATA().ticker_stoi(ticker)
src = dataset[idx].unsqueeze(0)[:, :-days_pred, :]
inference = inference_stonks_transformer(model, days_pred, src, device)
print(inference.shape)
ground_truth = HISTORY_DATA().array_to_dataframe(array=src[0,-days_pred:,:5].numpy())
inference = HISTORY_DATA().array_to_dataframe(array=inference[0].numpy())

print(ground_truth)
print(inference)
# out [ 0.0465, -0.2755, -0.3016,  0.9322, -0.5622, -0.8264,  0.8749, 0.5382]


src: tensor([[[-6.3795e-01, -6.3380e-01, -6.3896e-01,  ...,  2.0100e+03,
           6.0000e+00,  2.9000e+01],
         [-6.3326e-01, -6.3006e-01, -6.3498e-01,  ...,  2.0100e+03,
           6.0000e+00,  3.0000e+01],
         [-6.3380e-01, -6.3317e-01, -6.3707e-01,  ...,  2.0100e+03,
           7.0000e+00,  1.0000e+00],
         ...,
         [ 1.7118e+00,  1.7411e+00,  1.6620e+00,  ...,  2.0230e+03,
           6.0000e+00,  7.0000e+00],
         [ 1.6726e+00,  1.7867e+00,  1.6600e+00,  ...,  2.0230e+03,
           6.0000e+00,  8.0000e+00],
         [ 1.9301e+00,  1.9648e+00,  1.8570e+00,  ...,  2.0230e+03,
           6.0000e+00,  9.0000e+00]]], device='cuda:0') 
________________________
tgt: torch.Size([1, 1, 8]) 
 tensor([[-0.9201, -0.3916, -0.7350,  0.6781,  0.9223,  0.3864, -0.1171, -0.9931]],
       device='cuda:0') 
____________________________
src: torch.Size([1, 3260, 8]) 
 tensor([[-0.9201, -0.3916, -0.7350,  0.6781,  0.9223,  0.3864, -0.1171, -0.9931]],
       device='cuda:0') 


In [7]:
# TODO add some helpful data analysis and vizualization methods